In [1]:
%matplotlib widget
#%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import aseg_gdf2
from shapely.geometry import Point, LineString, MultiPoint, Polygon
from shapely.wkt import loads
from scipy.io import loadmat
from scipy import spatial
import matplotlib.pyplot as plt
import rasterio
import netCDF4
import h5py
import gc
import os
import glob
from scipy.spatial.ckdtree import cKDTree
from hydrogeol_utils import SNMR_utils, spatial_functions, AEM_utils
from hydrogeol_utils import plotting_utils as plot_utils

In [3]:
def nearest_neighbours(points, coords, points_required = 1, max_distance = 250.):
    """

    :param points: array of points to find the nearest neighbour for
    :param coords: coordinates of points
    :param points_required: number of points to return
    :param max_distance: maximum search radius
    :return:
    """
    # Initialise tree instance
    kdtree = cKDTree(data=coords)

    # iterate throught the points and find the nearest neighbour
    distances, indices = kdtree.query(points, k=points_required,
                                      distance_upper_bound=max_distance)
    print(distances)
    # Mask out infitnite distances in indices to avoid confusion
    mask = ~np.isfinite(distances)

    distances[mask] = np.nan

    return distances, indices

# Find the nearest neighbours within the maximum distance

def xy_2_var(grid_dict, xy, var):
    """
    Function for finding a variable for gridded AEM sections
    given an input easting and northing
    @ param: grid_dict :dictionary for gridded line data
    @ param: xy: numpy array with easting and northing
    @ param: var: string with variable name
    returns
    float: distance along line
    """
    utm_coords = np.column_stack((grid_dict['easting'],
                                  grid_dict['northing']))

    d, i = spatial_functions.nearest_neighbours(xy,
                                                utm_coords,
                                                points_required=1,
                                                max_distance=100.)
    if np.isnan(d[0]):
        return None

    else:
        near_ind = i[0]
    


        return grid_dict[var][near_ind]


In [4]:
infile = r"C:\Users\PCUser\Desktop\AEM\LCI\DalyR_WB_MGA52.nc"

lci_dat = netCDF4.Dataset(infile)

lci_coords = np.column_stack((lci_dat['easting'][:],
                          lci_dat['northing'][:]))

# Initialise tree instance for nearest neighbour searches
kdtree = cKDTree(data=lci_coords)


# bring in the rjmcmc data

infile = r"C:\Users\PCUser\Desktop\AEM\rjmcmc\DalyRiver_rjmcmc.nc"

# Read in the data

rj_dat = netCDF4.Dataset(infile)



In [5]:
# Since we will be wanting to plot the sections lets first grid them

# Create an instance of plots for gridding the data

plots = plot_utils.ConductivitySectionPlot(lci_dat)


# Define some key variables which we want to inteprolate

cond_vars = ['conductivity', 'data_residual', 'depth_of_investigation']

plots.conductivity_variables = cond_vars


# Define the resolution of th sections
xres, yres = 10., 2.


In [6]:
# We will use the lines from the rj

lines = rj_dat['line'][:]

# Now grid the lines and save in memory

hdf5_dir = r"C:\Users\PCUser\Desktop\NSC_data\data\AEM\DR\lci\hdf5"

if not os.path.exists(hdf5_dir):
    os.mkdir(hdf5_dir)

gridded_vars ={}

# Saved these out to save time

#gridded_vars = plots.grid_variables(xres = xres, yres =yres, lines=lines,
#                                    resampling_method = 'cubic', save_hdf5 = True,
 #                                   return_dict = True, hdf5_dir = hdf5_dir)

for line in lines:
    
    infile = os.path.join(hdf5_dir, str(line) + '.hdf5')
    f = h5py.File(infile, 'r')
    gridded_vars[line] = plot_utils.extract_hdf5_data(f, cond_vars)
    f = None
    gc.collect()

In [7]:
# Now we bring in the Oolloo Jinduckin contact to plot

#inRaster = r"C:\Users\PCUser\Desktop\NSC_data\data\raster\Daly_Aquifers\export\topOolloo_BOM.tif"
inRaster = r"C:\Users\PCUser\Desktop\NSC_data\data\raster\Daly_Aquifers\export\tJind_BOM.tif"

contact_dataset = rasterio.open(inRaster)

contact_elev = contact_dataset.read(1)

contact_elev[contact_elev == contact_dataset.get_nodatavals()] = np.nan


In [8]:
# We want to find the top of the Jinduckin for every gridded lci section

for line in gridded_vars.keys():
    
    # Get the coordinates
    utm_coords = utm_coords = np.column_stack((gridded_vars[line]['easting'],
                                               gridded_vars[line]['northing']))
    # sample the raster at the coordinates
    
    elevGen = contact_dataset.sample(utm_coords)
    
    gridded_vars[line]['topJind_elev'] = np.nan*np.ones(shape = utm_coords.shape[0],
                                                 dtype = np.float)
    
    for i in range(utm_coords.shape[0]):
        
        jindElev =  next(elevGen)

        if not np.isclose(jindElev,contact_dataset.get_nodatavals()[0]):
            gridded_vars[line]['topJind_elev'][i] = jindElev


In [9]:
# Now we bring in the magnetics to plot

inRaster = r"C:\Users\PCUser\Desktop\NSC_data\data\AEM\DR\2017_DalyRiver_SkyTEM\03_LCI\03_Depth_Slices\Grids_doi_Masked\*.ers"

cond = {}

for file in glob.glob(inRaster):
    layer = int(file.split('Con')[1].split('_')[0])
    cond_dataset = rasterio.open(file)
    cond[layer] = cond_dataset.read(1)



In [10]:
# Bring in a subset of the points to sample

infile = r"C:\Users\PCUser\Desktop\NSC_data\data\AEM\DR\garjmcmctdem_2\combined\rjmcmc_map.csv"

df = pd.read_csv(infile)



In [11]:
# generator for sampling the bom raster
contElev = contact_dataset.sample(df[['easting ', 'northing ']].values)

# Add the distance along the gridded lines to the dataframe

df['dist_along_line'] = np.nan

df['point_ind_rj'] = -9999

df['point_ind_lci'] = -9999

df['topJind_depth'] = np.nan

for index, row in df.iterrows():

    # get the line and coordinates
    line = row['line ']
    
    xy = np.array([row[['easting ','northing ']].values])
    
    df.loc[index,'dist_along_line'] = xy_2_var(gridded_vars[line],
                                               xy,'grid_distances')
    
    df.loc[index,'point_ind_rj'] = np.where(rj_dat['fiducial'][:]== row['fiducial '])[0][0]
    
    df.loc[index,'point_ind_lci'] = kdtree.query(row[['easting ', 'northing ']].values)[1]
    
    jindElev = next(contElev)[0]
    
    if not np.isclose(jindElev,contact_dataset.get_nodatavals()[0]):
        
        df.loc[index,'topJind_depth'] = row['elevation '] - jindElev

In [12]:
df_ss = df[df['topJind_depth'] < 400.]

In [15]:
df_ss['topJind_interp'] = np.nan

C:\Users\PCUser\Anaconda3\envs\hydrogeol_utils\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [13]:
# Create a generator to iterate through the pandas dataframe

def gen(df):
    for index, row in df.iterrows():
        yield index, row
        
#cond_gen = gen(df_rj_ss)


In [32]:
# This function stores the top of the conductor in the dataframe on a click
def onclick(event):
    if event.xdata != None and event.ydata != None:
        df_ss.at[ind, 'topJind_interp'] = event.ydata


def extract_data():
    """
    FUnction for extracting all the AEM data from the netCDF files
    """

    freq = rj_dat['conductivity_bin_count'][point_ind_rj].data.astype(np.float)

    cond_pdf = freq / freq.sum(axis =1)[0]

    cond_pdf[cond_pdf == 0] = np.nan
    
    cp_freq = rj_dat['change_point'][point_ind_rj].data.astype(np.float)
    
    cp_pdf = cp_freq / freq.sum(axis =1)[0]
    
    laybins = rj_dat['nlayer_bin_count'][point_ind_rj].data
    
    lay_prob = laybins / freq.sum(axis =1)[0]
    
    cond_cells = rj_dat["conductivity_cells"][:].data
    
    depth_cells = rj_dat['layer_top_depth'][point_ind_rj].data
    
    extent = [cond_cells.min(), cond_cells.max(), depth_cells.max(), depth_cells.min()]
    
    mean = rj_dat['conductivity_mean'][point_ind_rj].data
    p10 = rj_dat['conductivity_p10'][point_ind_rj].data
    p50 = rj_dat['conductivity_p50'][point_ind_rj].data
    p90 = rj_dat['conductivity_p90'][point_ind_rj].data
    
    lci_cond = lci_dat['conductivity'][point_ind_lci].data
    lci_depth_top = lci_dat['layer_top_depth'][point_ind_lci].data
    
    lci_doi = lci_dat['depth_of_investigation'][point_ind_lci].data
    
    misfit = rj_dat['misfit'][point_ind_rj].data
    sample_no = rj_dat['rj_sample_number'][:].data
    
    burnin = rj_dat["nburnin"][point_ind_rj].data
    nsamples = rj_dat['nsamples'][point_ind_rj].data
    nchains = rj_dat['nchains'][point_ind_rj].data
    
    line_ind = rj_dat['line_index'][point_ind_rj].data
    line = int(rj_dat['line'][line_ind].data)
    
    return {'conductivity_pdf': cond_pdf, "change_point_pdf": cp_pdf, "conductivity_extent": extent,
           'cond_p10': p10, 'cond_p50': p50, 'cond_p90': p90, 'cond_mean': mean, 'depth_cells': depth_cells,
           'nlayer_bins': laybins, 'nlayer_prob': lay_prob, 'nsamples': nsamples, 'ndata': rj_dat['n_data'][:].data,
           "nchains": nchains, 'burnin': burnin, 'misfit': misfit, 'sample_no': sample_no, 'cond_cells': cond_cells, 'lci_cond': lci_cond,
           'lci_depth_top': lci_depth_top, 'lci_doi': lci_doi, 'line': line}
    
def DR_plot(D, outfile = None):
    fig = plt.figure(figsize = (12,10))

    # These are for interactive widget mode
    fig.canvas.layout.width = '6in'
    fig.canvas.layout.height= '5in'

    ax1 = fig.add_axes([0.05, 0.35, 0.35, 0.62])
    ax2 = fig.add_axes([0.45, 0.35, 0.2, 0.62])
    ax3 = fig.add_axes([0.70, 0.52, 0.2, 0.2])
    ax4 = fig.add_axes([0.72, 0.32, 0.16, 0.16])
    ax5 = fig.add_axes([0.1, 0.18, 0.76, 0.05])
    ax6 = fig.add_axes([0.1, 0.05, 0.76, 0.13])
    ax7 = fig.add_axes([0.70, 0.78, 0.2, 0.2])
    cbar_ax1 = fig.add_axes([0.05, 0.29, 0.35, 0.01])
    cbar_ax2 = fig.add_axes([0.88, 0.05, 0.01, 0.2])
    
    panel_kwargs = [{'title': '',
                      'color': 'black',
                      'ylabel': 'data \n residual',
                      'legend': False},
                     {'title': 'LCI conductivity',
                      'max_depth': 450.,
                      'shade_doi': True,
                      'colourbar': True,
                      'colourbar_label': 'Conductivity (S/m)',
                      'log_plot': True,
                      'vmin': 0.001,
                      'vmax': 2.,
                      'cmap': 'jet',
                      'ylabel': 'elevation \n (mAHD)',
                      'vertical_exaggeration': 1.0}]


    # Plot probability map
    
    im = ax1.imshow(D['conductivity_pdf'], extent = D['conductivity_extent'],
                    aspect = 'auto', cmap = 'rainbow')
    
    #  PLot the median, and percentile plots
    ax1.plot(np.log10(D['cond_p10']), D['depth_cells'], c = 'k',linestyle='dashed', label = 'p10')
    ax1.plot(np.log10(D['cond_p90']), D['depth_cells'], c = 'k',linestyle='dashed', label = 'p90')
    ax1.plot(np.log10(D['cond_p50']), D['depth_cells'], c = 'k',label = 'p50')
    ax1.plot(np.log10(D['cond_mean']), D['depth_cells'], c = 'grey',label = 'mean')
    
    ax1.set_xticklabels([round(10 ** float(x), 4) for x in ax1.get_xticks()])

    # for lci layered model we do some processing
    lci_expanded = np.zeros(shape=2 * len(D['lci_cond']) + 1,
                                 dtype=np.float)

    lci_expanded[1:] = np.repeat(D['lci_cond'], 2)

    depth_expanded = (np.max(D['lci_depth_top']) + 10) * np.ones(shape=len(lci_expanded),
                                                            dtype=np.float)

    depth_expanded[:-1] = np.repeat(D['lci_depth_top'], 2)

    ax1.plot(np.log10(lci_expanded), depth_expanded, c = 'pink',
             linestyle = 'dashed', label = 'lci')
    ax1.plot(ax1.get_xlim(), [D['lci_doi'], D['lci_doi']], c = 'yellow',
             label = 'LCI doi')
    ax1.set_title('rj-MCMC probability map')
    ax1.set_ylabel('depth (mBGL)')
    ax1.set_xlabel('Conductivity (S/m)')
    ax1.grid(which = 'both')
    ax1.set_xlim(D['conductivity_extent'][0], D['conductivity_extent'][1] )
    
    ax1.set_ylim(D['conductivity_extent'][2], D['conductivity_extent'][3])#100.,0)#
    
    
    ax1.plot(ax1.get_xlim(), [row['topJind_depth'], row['topJind_depth']],
            c = 'green', linewidth = 2, label = 'top Jinduckin - BOM')
    ax1.legend(loc = 3)
    ax2.plot(D['change_point_pdf'], D['depth_cells'], label = 'P(change point)')
    ax2.set_ylim(ax2.get_ylim()[::-1])
    ax2.set_yticks(np.arange(0, 500, 20.))
    ax2.set_title('change point probability')
    ax2.set_ylim(D['conductivity_extent'][2], D['conductivity_extent'][3])#(100.,0)
    ax2.plot(ax2.get_xlim(), [row['topJind_depth'], row['topJind_depth']],
            c = 'green', linewidth = 2, label = 'top Jinduckin - BOM')
    ax2.legend()
    ax2.grid(which = 'both')
    
    
    

    ax3.imshow(contact_elev, 
               extent = [contact_dataset.bounds[0], contact_dataset.bounds[2],
                         contact_dataset.bounds[1], contact_dataset.bounds[3]],
              cmap= 'viridis')#, vmin = -2., vmax = 2.)
    #ax3.set_xlim(D['easting'] - 10000., D['easting'] + 10000.)
    #ax3.set_ylim(D['northing'] - 10000., D['northing'] + 10000.)
    ax3.plot(D['easting'],D['northing'], 'x', c = 'r')
    ax3.set_title('Oolloo-Jinduckin contact', fontsize=10)
    #ax3.set_title("Top Oolloo")
    ax3.tick_params(axis='both', which='major', labelsize=8)
    ax3.tick_params(axis='both', which='minor', labelsize=8)
    # Ax3 will be our location
    sample = D['sample_no'][:]
    
    # Add the misfit
    for i in range(D['misfit'].shape[0]):
       
        misfits = D['misfit'][i]
        ax4.plot(sample, misfits/D['ndata'])

    ax4.plot([1, D['nsamples']], [1,1], 'k')
    ax4.plot([D['burnin'], D['burnin']],[0.1,1e4], 'k')
    ax4.set_xlim([1, D['nsamples']])
    ax4.set_ylim(0.1, 1e4)

    ax4.set_xscale('log')
    ax4.set_yscale('log')

    ax4.set_xlabel("sample #")
    ax4.set_ylabel("Normalised misfit")
    
    # conductivity plot
    
    ax7.imshow(np.log10(cond[9]), extent = [cond_dataset.bounds[0],
                                  cond_dataset.bounds[2],
                                  cond_dataset.bounds[1], 
                                  cond_dataset.bounds[3]],
              cmap = 'jet',
              vmin = np.log10(panel_kwargs[1]['vmin']*1000.),
              vmax = np.log10(panel_kwargs[1]['vmax']*1000.))
    
    ax7.set_xlim(D['easting'] - 10000., D['easting'] + 10000.)
    ax7.set_ylim(D['northing'] - 10000., D['northing'] + 10000.)
    ax7.plot(D['easting'],D['northing'],  'x', c = 'k')
    
    p1 = [gridded_vars[line]['easting'][0], gridded_vars[line]['easting'][-1]]
    p2 = [gridded_vars[line]['northing'][0], gridded_vars[line]['northing'][-1]]
    ax7.plot(p1, p2, 'k', linewidth = 0.5)
    ax7.set_title('LCI depth slice 61.8-71.6 mBGL', fontsize=10)
    ax7.tick_params(axis='both', which='major', labelsize=8)
    ax7.tick_params(axis='both', which='minor', labelsize=8)
    cb1 = fig.colorbar(im, cax=cbar_ax1, orientation='horizontal')
    cb1.set_label('probabilitiy', fontsize=10)

    res1 = plot_utils.plot_single_line(ax5, gridded_vars[line],
                                       'data_residual', panel_kwargs[0])

    ax5.set_title('LCI conductivity section - ' + str(line))

    im2 = plot_utils.plot_grid(ax6, gridded_vars[line], 'conductivity',
                               panel_kwargs[1])
    
    # PLot jinduckin elevation onto section
    ax6.plot(gridded_vars[line]['grid_distances'],
            gridded_vars[line]['topJind_elev'], c = 'k',
            linestyle = 'dashed')
    
    # Add colorbars

    ax6.plot([dist, dist], [-500, 500], 'pink')
    
        
    cb2 = fig.colorbar(im2, cax=cbar_ax2, orientation='vertical')
    
    cb2.ax.set_yticklabels([round(10 ** x, 4) for x in cb2.get_ticks()])
    cb2.set_label('conductivity (S/m)', fontsize=10)
    
    ax5.set_xlim(dist,# - 5000.,
                 dist)# + 5000.)
    ax6.set_xlim(dist,# - 5000., 
                 dist)# + 5000.)

    
    
    return fig   

In [21]:
# 

df_ss.at[ind, 'topJind_interp']

255.01165511415832

In [40]:
# Define some coordinates to investigate
unid = 51

cond_gen = gen(df_ss[df['uniqueid '] == unid])


ind, row = next(cond_gen)

point_ind_lci = row['point_ind_lci']

point_ind_rj = row['point_ind_rj']

# Get the nearest lci point

easting, northing = row['easting '], row['northing ']


plt.close('all')

D = extract_data()

D['easting'], D['northing'] = easting, northing

line = row['line ']

# Find distance along the lci section
dist = xy_2_var(gridded_vars[line],
                 np.array([[easting, northing]]),
                         'grid_distances')


point_ind_lci= dist

fig = DR_plot(D)
#plt.savefig(r"C:\Users\PCUser\Desktop\NSC_data\DR_interp\rjplots\\Florina_" + row['BORE_NO'] + ".png", dpi = 150)
cid=  fig.canvas.mpl_connect('button_press_event', onclick)
plt.show()


C:\Users\PCUser\Anaconda3\envs\hydrogeol_utils\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

C:\Users\PCUser\Anaconda3\envs\hydrogeol_utils\lib\site-packages\ipykernel_launcher.py:116: RuntimeWarning: divide by zero encountered in log10
C:\Users\PCUser\Anaconda3\envs\hydrogeol_utils\lib\site-packages\ipykernel_launcher.py:178: RuntimeWarning: invalid value encountered in log10
C:\Users\PCUser\Anaconda3\envs\hydrogeol_utils\lib\site-packages\ipykernel_launcher.py:223: UserWarning: Attempting to set identical left == right == 10670.0 results in singular transformations; automatically expanding.
C:\Users\PCUser\Anaconda3\envs\hydrogeol_utils\lib\site-packages\ipykernel_launcher.py:225: UserWarning: Attempting to set identical left == right == 10670.0 results in singular transformations; automatically expanding.


In [94]:
# Here we extract some AEM data for further inversion

# bring in a shapefile as a geometry

infile = r"C:\Users\PCUser\Desktop\NSC_data\data\vector\DalyRiver\DalyGeology_August2018.shp"

from osgeo import ogr
file = ogr.Open(infile)
shape = file.GetLayer(0)
#first feature of the shapefile
feature = shape.GetFeature(3)

In [128]:
import json
shapefile = json.loads(feature.ExportToJson())

Oolloo_extent = Polygon(shapefile['geometry']['coordinates'][0])

from functools import partial
import pyproj
from shapely.ops import transform

project = partial(
    pyproj.transform,
    pyproj.Proj(init='epsg:4283'), # source coordinate system
    pyproj.Proj(init='epsg:28352')) # destination coordinate system

Oolloo_extent = transform(project, Oolloo_extent)  # apply projection

In [171]:
# bring in the inversion ready data

infile = r"C:\temp\DR_inversion_ready.dat"

gdf = aseg_gdf2.read(infile).df()

gdf = gdf[gdf['line '] > 102800]

gdf = gdf[gdf['line '] < 110302]

gdf = gdf[~gdf['line '].isin([105601, 105602, 106301, 109601, 109701, 110203,
                            110301, 109204, 109203])]

len(gdf)

174148

In [165]:
#Iterate through an get the indices of those that fall within the Oolloo

idx = []

for index, row in gdf.iterrows():
    coords = row[['easting ', 'northing']].values
    
    if Oolloo_extent.contains(Point(coords)):
        idx.append(index)

In [177]:

with open(infile, 'r' ) as inf:
    with open(infile.replace('.dat', '_Oolloo_ss.dat'), 'w') as outf:
        for i, line in enumerate(inf):
            if i in idx:
                outf.write(line)

In [173]:
row

' 1304 20170824 20170824.01  109803  1987689.50 835282.19 8332937.00    46.4   229.78   319.81  -0.68  -3.89   0.00 -13.20  -0.02  -2.90   1.303772E+03   6.575904E+02   3.666201E+02   2.966533E+02   2.163259E+02   1.643671E+02   1.213579E+02   8.753586E+01   6.320110E+01   4.364177E+01   2.932486E+01   2.019422E+01   1.243360E+01   7.912390E+00   4.841930E+00   2.799800E+00   1.637730E+00   9.954300E-01   2.078585E+01   1.751578E+01   1.445528E+01   1.158486E+01   9.019670E+00   6.797270E+00   4.939920E+00   3.482580E+00   2.357370E+00   1.526070E+00   9.537500E-01   5.677100E-01   3.246600E-01   1.810300E-01   9.784000E-02   4.896000E-02   2.466000E-02   1.280000E-02   6.360000E-03   2.520000E-03   9.700000E-04   3.900000E-04  -1.500000E-04   4.008152E+01   2.156813E+01   1.259915E+01   9.489408E+00   7.013184E+00   5.332332E+00   3.967027E+00   2.920324E+00   2.161076E+00   1.551778E+00   1.063017E+00   7.623128E-01   5.074975E-01   3.602310E-01   2.744778E-01   2.102015E-01   1.6762

In [174]:
i in idx

False

In [176]:
idx[66] in idx

True